# Five ways to create NiceCX networks:

- Starting with an Empty Network
- CX Files
- NDEx Networks
- NetworkX Networks
- Pandas DataFrames

In this tutorial you will learn to use NiceCX, a simple data model that is part of the ndex2 NDEx Client module.
NiceCX facilitates creating and working with networks, including interfaces to NetworkX and Pandas.
This tutorial requires Python 3.6+ and the ndex2 module, see the NDEx Client Tutorial for installation instructions.



### Import Packages Required for this Tutorial

In [ ]:
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
import networkx as nx
import pandas as pd
import os

For information on **how to access nodes, edges, attributes, etc...** see the end of this notebook ([click here](#first-bullet))

## Starting with an Empty Network

**Create an empty niceCx network**

In [ ]:
nice_cx_creatures = NiceCXNetwork()

**Name the Network**

The _niceCx_creatures_ will now be populated with data in which each node represents a species and has a color attribute. Each edge will specify a relationship between the two species. First, we will set the name of the network:  

In [ ]:
nice_cx_creatures.set_name("Food Web")

**Add Nodes and Edges**

In [ ]:
fox_node = nice_cx_creatures.create_node(node_name='Fox')
mouse_node = nice_cx_creatures.create_node(node_name='Mouse')
bird_node = nice_cx_creatures.create_node(node_name='Bird')
fox_bird_edge = nice_cx_creatures.create_edge(edge_source=fox_node, edge_target=bird_node, edge_interaction='interacts-with')

fox_mouse_edge = nice_cx_creatures.create_edge(edge_source=fox_node, edge_target=mouse_node, edge_interaction='interacts-with')

print('The fox node has id: %d' % fox_node)

print('The mouse node has id: %d' % mouse_node)


The create_node() and create_edge() methods return the unique ID assigned to the new node or edge. In CX, IDs are always assigned in an ascending order, although they may not always be sequential In this case, the node with the name "Fox" will have an ID of 0, "Mouse" will have 1, and so on. The edge between "Fox" and "Bird" will have the ID of 0, the next will be 1, and so on. 

**Add Attributes**

The add_node_attribute() and add_edge_attribute() methods require the ID in the "property_of" field and the property "name" and "values" in their respective fields.

In [ ]:
print(nice_cx_creatures)
nice_cx_creatures.add_node_attribute(property_of=fox_node, name='Color', values='Red')

nice_cx_creatures.add_node_attribute(property_of=mouse_node, name='Color', values='Gray')

nice_cx_creatures.add_node_attribute(property_of=bird_node, name='Color', values='Blue')

In [ ]:
nice_cx_creatures.add_edge_attribute(property_of=fox_mouse_edge, name='Hunted', values='On the ground')

We can now print a summary of niceCX_creatures. The returned value of the getSummary() method is structured in the same format as NetworkSummary objects returned by NDEx network search methods.

In [ ]:
nice_cx_creatures.print_summary()

## CX Files

In [ ]:
nice_cx_from_cx_file = ndex2.create_nice_cx_from_file('SimpleNetwork.cx')

nice_cx_from_cx_file.print_summary()

## NDEx Networks

In [ ]:
nice_cx_from_server = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')

nice_cx_from_server.print_summary()

## NetworkX Networks

Networks can be manipulated using NetworkX facilities and then used to create a NiceCx network. Here we create a NetworkX network from scratch:

In [ ]:
G = nx.Graph()
G.add_node('ABC')
G.add_node('DEF')
G.add_node('GHI')
G.add_node('JKL')
G.add_node('MNO')
G.add_node('PQR')
G.add_node('XYZ')
G.add_edges_from([('ABC','DEF'), ('DEF', 'GHI'),('GHI', 'JKL'), 
                  ('DEF', 'JKL'), ('JKL', 'MNO'), ('DEF', 'MNO'),
                 ('MNO', 'XYZ'), ('DEF', 'PQR')])

We now use NetworkX to find the shortest path between nodes ABC and MNO and then create a subgraph from that path.

In [ ]:
short_path = nx.shortest_path(G,source='ABC',target="MNO")

path_subgraph = G.subgraph(short_path)

Finally, NiceCx objects are created from both *G* and *path_subgraph*.

In [ ]:
G.name = 'Created from NetworkX (full)'
nice_cx_full = ndex2.create_nice_cx_from_networkx(G)

G.name = 'Created from NetworkX (shortest path)'
nice_cx_short = ndex2.create_nice_cx_from_networkx(path_subgraph)
                       
nice_cx_full.print_summary()
print(G.edges())
print('')

nice_cx_short.print_summary()
print(path_subgraph.edges())

## Pandas DataFrames

### 2 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF'), ('DEF', 'XYZ')]

df = pd.DataFrame.from_records(data)

nice_cx_df_2_column = ndex2.create_nice_cx_from_pandas(df)

nice_cx_df_2_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_2_col_from_niceCx = nice_cx_df_2_column.to_pandas_dataframe()
print(df_2_col_from_niceCx)

### 3 Column DataFrame with No Headers:

In [ ]:
data = [('ABC', 'DEF', 'interacts-with'), ('DEF', 'XYZ', 'neighbor-of')]

df = pd.DataFrame.from_records(data)

nice_cx_df_3_column = ndex2.create_nice_cx_from_pandas(df)

nice_cx_df_3_column.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_3_col_from_niceCx = nice_cx_df_3_column.to_pandas_dataframe()
print(df_3_col_from_niceCx)

### 3+ Column DataFrame with Headers to Specify Attribute Columns

In [ ]:
df = pd.DataFrame.from_dict(dict([('Source', ['ABC', 'DEF']),
                              ('Target', ['DEF', 'XYZ']),
                              ('Interaction', ['interacts-with', 'neighbor-of']),
                              ('EdgeProp', ['Edge property 1', 'Edge property 2'])])

nice_cx_df_with_headers = ndex2.create_nice_cx_from_pandas(df, source_field='Source', target_field='Target', 
                          edge_attr=['EdgeProp'], edge_interaction='Interaction')

nice_cx_df_with_headers.print_summary()

#===========================
# CONVERT BACK TO DATAFRAME
#===========================
df_headers_from_niceCx = nice_cx_df_with_headers.to_pandas_dataframe()
print(df_headers_from_niceCx)

### Exporting a Larger and More Complex Network

More complex networks can be output to a Pandas DataFrame as well. In the following example we convert the example network we loaded from the server.

In [ ]:
#======================
# CONVERT TO DATAFRAME
#======================
nice_cx_from_server_df = nice_cx_from_server.to_pandas_dataframe()
print(nice_cx_from_server_df)

Likewise, networks can be output as a networkx graph.

In [ ]:
#=====================
# CONVERT TO NETWORKX
#=====================
nice_cx_from_server_to_netx = nice_cx_from_server.to_networkx(mode='default')

print(nice_cx_from_server_to_netx)

**To continue with the tutorial, you must edit the following cell to replace the values of the ‘my_account’ and ‘my_password’ variables with a real NDEx account name and password.**

In [ ]:
my_account="enter your username here"
my_password="enter your password here"
my_server="http://public.ndexbio.org"

if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    try:
        my_ndex=nc.Ndex2(my_server, my_account, my_password)
        my_ndex.update_status()
        print("Success.  Please continue.")
    except Exception as inst:
        print("Could not access account %s with password %s" % (my_account, my_password))
        print(inst.args)

NiceCx networks can be saved to the NDEx server by calling **upload_to()**

In [ ]:
if my_account == 'enter your username here':
    print('*******WARNING!!!!*******')
    print('Please change the username and password before proceeding')
else:
    upload_message = nice_cx_df_with_headers.upload_to(my_server, my_account, my_password)
    print(upload_message)

## Accessing nodes, edges, attributes, etc... <a class="anchor" id="first-bullet"></a>

### Nodes

In [ ]:
# EXAMPLE NETWORK FROM NDEx

nice_cx_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid='f1dd6cc3-0007-11e6-b550-06603eb7f303')

Loop through all **nodes** and print when we reach the RAD52 node

In [ ]:
for node_id, node in nice_cx_network.get_nodes():
    if node.get('n') == 'RAD52':
        print(node)
        break

Nodes and edges are indexed by id (@id from the example above).  To look up a specific node by it's _name_ you can create a reverse look up index as follows

In [ ]:
#============================
# CREATE A NAME to ID LOOKUP 
#============================
node_name_lookup = {node.get('n'): node.get('@id') for node_id, node in nice_cx_network.get_nodes()}

#========================
# GET THE 'MAP2K1' NODE
#========================
map2k1_node_id = node_name_lookup.get('MAP2K1')
map2k1_node = nice_cx_network.get_node( map2k1_node_id )
print(map2k1_node)

### Node attributes

Get the node attribute (Pathway) for MAP2K1

In [ ]:
map2k1_pathway_attribute = nice_cx_network.get_node_attribute(map2k1_node_id, 'Pathway')
print(map2k1_pathway_attribute)

### Edges

Print all the **edges** where MAP2K1 is either a source or target

In [ ]:
map2k1_edges = []

for edge_id, edge in nice_cx_network.get_edges():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        map2k1_edges.append(edge.get('@id'))

        print(edge)


Or, if you would like to see the node labels represented in these edges you can look up and print the node name

In [ ]:
for edge_id, edge in nice_cx_network.get_edges():
    if edge.get('s') == map2k1_node_id or edge.get('t') == map2k1_node_id:
        source_node = nice_cx_network.get_node( edge.get('s') )
        target_node = nice_cx_network.get_node( edge.get('t') )
        print('Source: %s Target: %s Interaction: %s' % (source_node.get('n'), target_node.get('n'), edge.get('i')))

### Edge attributes

Print the **edge attributes** (yeastSscore) for MAP2K1 edges

In [ ]:
for edge_id in map2k1_edges:
    print(nice_cx_network.get_edge_attribute(edge_id, 'yeastSscore'))


### Network attributes

Print the reference attribute for this network

In [ ]:
nice_cx_network.get_network_attribute('reference')

Or print the html content in the value field

In [ ]:
from IPython.core.display import display, HTML

display(HTML(nice_cx_network.get_network_attribute('reference').get('v')))